In [73]:
import requests
import pandas as pd
import csv


# travelMode can be either driving, walking, or transit
travelMode = 'driving'
  
# timeUnit can be minute or second
timeUnit = 'minute'
    
# HTTP POST Request URL
url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AtWNtrHV5F1mwktqPg7LB8iV1On-Qt8YLhjdqHbI_Qlm4AG-UM_7-qwz7b0n812Q'
 
    
# -----------------------------------------------
# Building the origins and destinations table
# -----------------------------------------------

# Reading in all the locations

locations = {}
ids = []
with open('../../Milwaukee Case Study Raw Data/potential_db_locations.csv', 'r', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    count = 0
    for row in spamreader:
        if count > 0:
            row[1] = float(row[1])
            row[2] = float(row[2])
            locations[(row[1],row[2])] = row[0]
            ids.append(row[0])
        else: 
            count = 1

locations_latlong = list(locations.keys())
# -----------------------------------------------
# Requesting the distance
# -----------------------------------------------
distance_matrix = {}
duration_matrix = {}

# Need to split into quarters as to not exceed limit

for h1 in [0,1]: # Which half for the origins
    for h2 in [0,1]: # Which half for the destinations
        
        # Creating the POST body
        bod = {
        "origins": [{'latitude': x[0], "longitude": x[1]} for x in locations_latlong[50*h1:50+h1*44]],
        "destinations": [{'latitude': x[0], "longitude": x[1]} for x in locations_latlong[50*h2:50+h2*44]],
        "travelMode": travelMode,
        "timeUnit": timeUnit,
        'distanceUnit': 'mile' 
        }
        
        
        r = requests.post(url, data=json.dumps(bod))
        travels = r.json()['resourceSets'][0]['resources'][0]['results']
        origins =  r.json()['resourceSets'][0]['resources'][0]['origins']
        destinations =  r.json()['resourceSets'][0]['resources'][0]['destinations']
        
        # Adding to the distance matrix
        for diction in travels:
            o = origins[diction['originIndex']]
            d = destinations[diction['destinationIndex']]
            o_ID = locations[o['latitude'],o['longitude']]
            d_ID = locations[d['latitude'],d['longitude']]
            distance_matrix[o_ID,d_ID] = diction['travelDistance']
            duration_matrix[o_ID,d_ID] = diction['travelDuration']

with open(f'dropbox_{travelMode}_distance_matrix.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["ID (From\To)"] + ids)
    for i in ids:
        spamwriter.writerow([i]+[distance_matrix[i,j] for j in ids])

with open(f'dropbox_{travelMode}_duration_matrix.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["ID (From\To)"] + ids)
    for i in ids:
        spamwriter.writerow([i]+[duration_matrix[i,j] for j in ids])

In [74]:
distance_matrix

{('Court1', 'Court1'): 0,
 ('Court1', 'Court2'): 0.2094,
 ('Court1', 'Court3'): 0.1299,
 ('Court1', 'Court4'): 1.3913,
 ('Court1', 'Elect1'): 1.2024,
 ('Court1', 'Elect2'): 2.7514,
 ('Court1', 'Fire1'): 1.1881,
 ('Court1', 'Fire2'): 0.5145,
 ('Court1', 'Fire4'): 14.2331,
 ('Court1', 'Fire6'): 1.9617,
 ('Court1', 'Fire7'): 5.4706,
 ('Court1', 'Fire8'): 9.8736,
 ('Court1', 'Fire9'): 9.0552,
 ('Court1', 'Fire10'): 6.9786,
 ('Court1', 'Fire11'): 3.942,
 ('Court1', 'Fire12'): 6.374,
 ('Court1', 'Fire13'): 2.7434,
 ('Court1', 'Fire14'): 9.6039,
 ('Court1', 'Fire16'): 15.6057,
 ('Court1', 'Fire18'): 3.7928,
 ('Court1', 'Fire21'): 2.1835,
 ('Court1', 'Fire22'): 7.9529,
 ('Court1', 'Fire23'): 2.3339,
 ('Court1', 'Fire24'): 6.6965,
 ('Court1', 'Fire26'): 2.7757,
 ('Court1', 'Fire27'): 4.0924,
 ('Court1', 'Fire29'): 10.9063,
 ('Court1', 'Fire30'): 2.7359,
 ('Court1', 'Fire32'): 2.0188,
 ('Court1', 'Fire33'): 4.7939,
 ('Court1', 'Fire34'): 6.3759,
 ('Court1', 'Fire35'): 4.8305,
 ('Court1', 'Fire36

In [82]:
import requests
import pandas as pd
import csv
import json

# travelMode can be either driving, walking, or transit
travelMode = 'transit'
  
# timeUnit can be minute or second
timeUnit = 'minute'
    
# HTTP POST Request URL
url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AtWNtrHV5F1mwktqPg7LB8iV1On-Qt8YLhjdqHbI_Qlm4AG-UM_7-qwz7b0n812Q'
 
    
# -----------------------------------------------
# Building the origins of voter locations
# -----------------------------------------------

# Reading in all the locations

voter_locations = {}
voter_ids = []
voters_latlong = []
with open('../../Milwaukee Case Study Raw Data/votingWardCenters.csv', 'r', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    count = 0
    for row in spamreader:
        if count > 0:
            row[-2] = float(row[-2]) # lat
            row[-1] = float(row[-1]) # long
            voter_locations[(row[-2],row[-1])] = row[1] # ward
            voter_ids.append(row[1])
            voters_latlong.append((row[-2],row[-1]))
        else: 
            count = 1

# -----------------------------------------------
# Building the destinations (drop box locations)
# -----------------------------------------------
db_locations = {}
db_ids = []
db_latlong = []
with open('../../Milwaukee Case Study Raw Data/potential_db_locations.csv', 'r', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    count = 0
    for row in spamreader:
        if count > 0:
            row[1] = float(row[1]) # lat
            row[2] = float(row[2]) # long
            db_locations[(row[1],row[2])] = row[0] # ward
            db_ids.append(row[0])
            db_latlong.append((row[1],row[2]))
        else: 
            count = 1

# -----------------------------------------------
# Requesting the distance
# -----------------------------------------------
distance_matrix = {}
duration_matrix = {}

# Need to split into quarters as to not exceed limit
if travelMode == 'driving': # Up to 50,000 pairs per call
    h1_list = [0,1,2,3,4,5,6]
    h2_list = [0,1]
    h1_mult = 50
    h2_mult = 50
    
else: # Up to 625 pairs per call
    h1_list = range(22)
    h2_list = range(4)
    h1_mult = 15
    h2_mult = 25
    
    
for h1 in h1_list: # Which nth of the voter locations
    for h2 in h2_list: # Which nth for the destinations
        
        # Creating the POST body
        bod = {
        "origins": [{'latitude': x[0], "longitude": x[1]} for x in voters_latlong[h1_mult*h1:h1_mult*(h1+1)]],
        "destinations": [{'latitude': x[0], "longitude": x[1]} for x in db_latlong[h2_mult*h2:h2_mult*(h2+1)]],
        "travelMode": travelMode,
        "timeUnit": timeUnit,
        'distanceUnit': 'mile' 
        }
        
        
        r = requests.post(url, data=json.dumps(bod))
        travels = r.json()['resourceSets'][0]['resources'][0]['results']
        origins =  r.json()['resourceSets'][0]['resources'][0]['origins']
        destinations =  r.json()['resourceSets'][0]['resources'][0]['destinations']
        
        # Adding to the distance matrix
        for diction in travels:
            o = origins[diction['originIndex']]
            d = destinations[diction['destinationIndex']]
            o_ID = voter_locations[o['latitude'],o['longitude']]
            d_ID = db_locations[d['latitude'],d['longitude']]
            distance_matrix[o_ID,d_ID] = diction['travelDistance']
            duration_matrix[o_ID,d_ID] = diction['travelDuration']
            
with open(f'voter_{travelMode}_distance_matrix.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["ID (From\To)"] + db_ids)
    for i in voter_ids:
        spamwriter.writerow([i]+[distance_matrix[i,j] for j in db_ids])

with open(f'voter_{travelMode}_duration_matrix.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["ID (From\To)"] + db_ids)
    for i in voter_ids:
        spamwriter.writerow([i]+[duration_matrix[i,j] for j in db_ids])

In [78]:
r.json()


{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2021 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.',
 'errors': [{'errorCode': '',
   'errorDetails': ['Maximum 625 pairs allowed for transit matrix.']}],
 'resourceSets': [],
 'statusCode': 400,
 'statusDescription': 'Maximum 625 pairs allowed for transit matrix.',
 'traceId': '3d664b8dd8eb413ab239e0947c94c9f3|CH0001B06C|0.0.0.0|CH0000204D'}

In [84]:
duration_matrix

{('128', 'Fire6'): 10,
 ('128', 'Elect1'): 18,
 ('128', 'Fire10'): 51,
 ('128', 'Court2'): 26,
 ('129', 'Fire12'): 49,
 ('129', 'Fire8'): 49,
 ('126', 'Fire22'): 46,
 ('126', 'Court1'): 27,
 ('124', 'Court4'): 24,
 ('124', 'Fire14'): 59,
 ('127', 'Fire8'): 40,
 ('246', 'Fire6'): 30,
 ('246', 'Fire16'): 85,
 ('202', 'Fire16'): 56,
 ('202', 'Fire22'): 36,
 ('202', 'Court1'): 16,
 ('202', 'Fire2'): 15,
 ('199', 'Fire24'): 28,
 ('199', 'Fire4'): 35,
 ('126', 'Fire14'): 53,
 ('126', 'Fire24'): 30,
 ('125', 'Elect1'): 20,
 ('125', 'Fire10'): 53,
 ('242', 'Elect1'): 21,
 ('242', 'Elect2'): 11,
 ('246', 'Fire9'): 63,
 ('200', 'Fire16'): 51,
 ('200', 'Fire22'): 34,
 ('200', 'Court1'): 14,
 ('200', 'Fire2'): 14,
 ('199', 'Court2'): 11,
 ('199', 'Fire21'): 26,
 ('125', 'Fire2'): 24,
 ('127', 'Fire23'): 36,
 ('242', 'Fire23'): 18,
 ('242', 'Fire26'): 23,
 ('242', 'Fire14'): 29,
 ('242', 'Fire11'): 6,
 ('242', 'Fire16'): 85,
 ('202', 'Court4'): 26,
 ('202', 'Fire14'): 50,
 ('202', 'Fire24'): 26,
 (

In [92]:
voter_locations

{(43.09026251, -87.904165): '124',
 (43.07718535, -87.90006458): '125',
 (43.07656721, -87.89518985): '126',
 (43.07804625, -87.88806632): '127',
 (43.071036, -87.88836773): '128',
 (43.07083352, -87.88067721): '129',
 (43.00017159, -87.89479039): '242',
 (42.99609857, -87.88784579): '243',
 (42.99183521, -87.88265927): '244',
 (42.98932581, -87.88091839): '245',
 (42.99235413, -87.89759325): '246',
 (43.04376628, -87.95289335): '199',
 (43.04748299, -87.95328896): '200',
 (43.04359697, -87.96198539): '201',
 (43.04407411, -87.96949777): '202',
 (43.04712567, -87.97852496): '203',
 (43.05063906, -87.98358091): '204',
 (43.04011394, -87.98026833): '205',
 (43.03469077, -87.97776771): '206',
 (43.05260839, -87.93156922): '174',
 (43.05462079, -87.91868071): '175',
 (43.04960252, -87.91782605): '176',
 (43.05721968, -87.90630202): '177',
 (43.05432167, -87.8983101): '178',
 (43.05714727, -87.89187098): '179',
 (43.05701898, -87.88357355): '180',
 (43.03428807, -87.90302555): '186',
 (43.0

In [90]:
duration_matrix['318','Court1']

0

In [93]:
import requests
import pandas as pd
import csv
import json

# travelMode can be either driving, walking, or transit
travelMode = 'walking'
  
# timeUnit can be minute or second
timeUnit = 'minute'
    
# HTTP POST Request URL
url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AtWNtrHV5F1mwktqPg7LB8iV1On-Qt8YLhjdqHbI_Qlm4AG-UM_7-qwz7b0n812Q'
 
bod = {
    "origins": [{
        "latitude": 43.19367388,
        "longitude": -88.06478537
    }],
    "destinations": [{
        "latitude": 43.04177537,
        "longitude": -87.92396736}
    ],
    "travelMode": "transit"
}
r = requests.post(url, data=json.dumps(bod))
r.json()

{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2021 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.',
 'resourceSets': [{'estimatedTotal': 1,
   'resources': [{'__type': 'DistanceMatrix:http://schemas.microsoft.com/search/local/ws/rest/v1',
     'destinations': [{'latitude': 43.04177537, 'longitude': -87.92396736}],
     'origins': [{'latitude': 43.19367388, 'longitude': -88.06478537}],
     'results': [{'destinationIndex': 0,
       'originIndex': 0,
       'totalWalkDuration': 0,
       'travelDistance': 0,
       'travelDuration': 0}]}]}],
 'statusCode': 200,
 'statusDescription': 'OK',
 'traceId': '97f346014da44b1ab5fd8fd7fd430a13|CH0001B060|0.0.0.0|CH000014BE'}

In [10]:
import requests
import pandas as pd
import csv
import json
import math

# Which location are we making it for?
loc = 'Mequon' # 'Milwaukee' #

# travelMode can be either driving, walking, or transit
travelMode = 'driving'
  
# timeUnit can be minute or second
timeUnit = 'minute'
    
# HTTP POST Request URL
url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AtWNtrHV5F1mwktqPg7LB8iV1On-Qt8YLhjdqHbI_Qlm4AG-UM_7-qwz7b0n812Q'
 
    
# -----------------------------------------------
# Building the origins of voter locations
# -----------------------------------------------

# Reading in all the locations

voter_locations = {}
voter_ids = []
voters_latlong = []
with open(f'../../{loc} Case Study Raw Data/votingWardCenters.csv', 'r', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    count = 0
    for row in spamreader:
        if count > 0:
            row[-2] = float(row[-2]) # lat
            row[-1] = float(row[-1]) # long
            voter_locations[(row[-2],row[-1])] = row[1] # ward
            voter_ids.append(row[1])
            voters_latlong.append((row[-2],row[-1]))
        else: 
            count = 1

# -----------------------------------------------
# Building the destinations (drop box locations)
# -----------------------------------------------
db_locations = {}
db_ids = []
db_latlong = []
with open(f'../../{loc} Case Study Raw Data/potential_db_locations.csv', 'r', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    count = 0
    for row in spamreader:
        if count > 0:
            row[1] = float(row[1]) # lat
            row[2] = float(row[2]) # long
            db_locations[(row[1],row[2])] = row[0] # ward
            db_ids.append(row[0])
            db_latlong.append((row[1],row[2]))
        else: 
            count = 1

# -----------------------------------------------
# Requesting the distance
# -----------------------------------------------
distance_matrix = {}
duration_matrix = {}

# Need to split into quarters as to not exceed limit
if travelMode == 'driving': # Up to 50,000 pairs per call
    h1_mult = 50
    h2_mult = 50
    h1_list = range(math.ceil(len(voter_ids)/h1_mult))
    h2_list = range(math.ceil(len(db_ids)/h2_mult))
    
    
else: # Up to 675 pairs per call
    h1_mult = 15
    h2_mult = 25
    h1_list = range(math.ceil(len(voter_ids)/h1_mult))
    h2_list = range(math.ceil(len(db_ids)/h2_mult))
    
    
for h1 in [0,1,2,3,4,5,6]: # Which 7th of the voter locations
    for h2 in [0,1]: # Which half for the destinations
        
        # Creating the POST body
        bod = {
        "origins": [{'latitude': x[0], "longitude": x[1]} for x in voters_latlong[h1_mult*h1:h1_mult*(h1+1)]],
        "destinations": [{'latitude': x[0], "longitude": x[1]} for x in db_latlong[h2_mult*h2:h2_mult*(h2+1)]],
        "travelMode": travelMode,
        "timeUnit": timeUnit,
        'distanceUnit': 'mile' 
        }
        
        
        r = requests.post(url, data=json.dumps(bod))
        travels = r.json()['resourceSets'][0]['resources'][0]['results']
        origins =  r.json()['resourceSets'][0]['resources'][0]['origins']
        destinations =  r.json()['resourceSets'][0]['resources'][0]['destinations']
        
        # Adding to the distance matrix
        for diction in travels:
            o = origins[diction['originIndex']]
            d = destinations[diction['destinationIndex']]
            o_ID = voter_locations[o['latitude'],o['longitude']]
            d_ID = db_locations[d['latitude'],d['longitude']]
            distance_matrix[o_ID,d_ID] = diction['travelDistance']
            duration_matrix[o_ID,d_ID] = diction['travelDuration']

KeyError: (43.21390281, -87.91421061)